In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import timm
from torch.utils.data import DataLoader
import time

# ImageNet Sketch dataset (Wang, Ge et al., 2019)

In [ ]:
# Download ImageNet-Sketch from Google Drive
!pip install -q -U gdown
!gdown --id 1Mj0i5HBthqH1p_yeXzsg22gZduvgoNeA

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1Mj0i5HBthqH1p_yeXzsg22gZduvgoNeA
From (redirected): https://drive.google.com/uc?id=1Mj0i5HBthqH1p_yeXzsg22gZduvgoNeA&confirm=t&uuid=47d49bf1-63c6-4aa0-bd8c-c119b21b1afd
To: /content/ImageNet-Sketch.zip
100% 7.59G/7.59G [01:25<00:00, 88.8MB/s]


In [ ]:
!unzip -q ImageNet-Sketch.zip -d .

data_dir = "sketch"

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Assuming the extracted data is in a folder named 'sketch'
sketch_dataset = datasets.ImageFolder(data_dir, transform=transform)
sketch_loader = DataLoader(sketch_dataset, batch_size=64, shuffle=False, num_workers=2)

print("Classes:", len(sketch_dataset.classes))
print("Total images:", len(sketch_dataset))

Classes: 1000
Total images: 50889


3 pre-trained models on ImageNet from Torchvision

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

models = {
    "ResNet50": torchvision.models.resnet50(weights='IMAGENET1K_V1').to(device).eval(),
    "ViT-B/16": torchvision.models.vit_b_16(weights='IMAGENET1K_V1').to(device).eval(),
    "EffNet-B3": torchvision.models.efficientnet_b3(weights='IMAGENET1K_V1').to(device).eval(),
}

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 153MB/s]


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth


100%|██████████| 330M/330M [00:05<00:00, 65.9MB/s]


Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-b3899882.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-b3899882.pth


100%|██████████| 47.2M/47.2M [00:00<00:00, 165MB/s]


In [ ]:
from tqdm.auto import tqdm

@torch.no_grad()
def evaluate(model, loader):
    correct, total = 0, 0
    for imgs, labels in tqdm(loader, desc="Evaluating"):
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    return correct / total

In [ ]:
results = {}
for name, model in models.items():
    print(f"Evaluating {name}...")
    acc = evaluate(model, sketch_loader)
    results[name] = acc
    print(f"{name}: {acc*100:.2f}%\n")

Evaluating ResNet50...


Evaluating:   0%|          | 0/796 [00:00<?, ?it/s]

ResNet50: 24.09%

Evaluating ViT-B/16...


Evaluating:   0%|          | 0/796 [00:00<?, ?it/s]

ViT-B/16: 29.41%

Evaluating EffNet-B3...


Evaluating:   0%|          | 0/796 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x79a8805c5260>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x79a8805c5260>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 16

EffNet-B3: 34.09%



# Base Model performance

To evaluate these performances against their base result for ImageNet dataset, I took the accuracies as reported for ImageNet-1K from PyTorch documentation [[link]](https://docs.pytorch.org/vision/main/models.html#table-of-all-available-classification-weights).

In [ ]:
imagenet_baseline = {
    "ResNet50": 76.13,
    "ViT-B/16": 81.072,
    "EffNet-B3": 82.008,
}

print("\n--- Results ---")
for name in results:
    drop = imagenet_baseline[name] - results[name]*100
    print(f"{name}: Sketch Acc = {results[name]*100:.2f}%, "
          f"Baseline = {imagenet_baseline[name]:.1f}%, "
          f"Drop = {drop:.2f}%")


--- Results ---
ResNet50: Sketch Acc = 24.09%, Baseline = 76.1%, Drop = 52.04%
ViT-B/16: Sketch Acc = 29.41%, Baseline = 81.1%, Drop = 51.66%
EffNet-B3: Sketch Acc = 34.09%, Baseline = 82.0%, Drop = 47.92%


**For each of these models, to what extent does the performance drop due to the shift in distribution?**

Looking at these performance numbers as compared to base accuracies from PyTorch documentation, all of the models had roughly $\approx50\%$ drop in the accuracies.